<a href="https://colab.research.google.com/github/NJain121442/course2020/blob/master/16August_SEM_Conv2d_V2_w/o_FC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [169]:
!pip install kornia
from kornia import filters
import torch.nn as nn
import glob
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn.functional as F
import torch.optim as optim
from IPython.display import clear_output
from torchvision import datasets, transforms
import numpy as np
from torchsummary import summary 
import matplotlib.image as mpimg
import torchvision
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
from PIL import Image

In [170]:
from google.colab import drive 
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [172]:
#Data_path = "/content/Fruits/"
#Data_path = "/content/drive/My Drive/AutoEncoder_training/AutoEncoder_training/"
Data_path = "/content/drive/My Drive/All_class_images"

Lets Resize all images to same size

In [173]:
class CustomDataset(Dataset):
  def __init__(self,path,new_size = 226):
    #self.filelist = glob.glob(path+'/*.jfif')
    self.filelist = glob.glob(path+'/*.tiff')
    self.new_size = new_size

  def __len__(self):
    return len(self.filelist)

  def __getitem__(self, idx):
    temp_filepath = self.filelist[idx]
    temp_image = cv2.imread(temp_filepath)
    temp_image = cv2.cvtColor(temp_image, cv2.COLOR_BGR2GRAY)
    temp_image = temp_image/255
    temp_image = torch.Tensor(temp_image)
    temp_image = temp_image.unsqueeze(0)
    return temp_image

In [174]:
Data = CustomDataset(Data_path)
Data[2].shape

torch.Size([1, 504, 504])

In [176]:
class COV2D_AutoEncoder(nn.Module):
    def __init__(self,filters = [64,128,256,512]):
        super(COV2D_AutoEncoder,self).__init__()
        self.enc1 = nn.Conv2d(1,filters[0],kernel_size=3,stride=3)
        self.enc2 = nn.Conv2d(filters[0],filters[1],kernel_size=3,stride=3)
        self.enc3 = nn.Conv2d(filters[1],filters[2],kernel_size=2,stride=3)
        self.enc4 = nn.Conv2d(filters[2],filters[3],kernel_size=3,stride=2)
        self.pool = nn.MaxPool2d(2,2,return_indices=True)
        self.dec1 = nn.ConvTranspose2d(filters[3],filters[2],kernel_size=3,stride=2)
        self.dec2 = nn.ConvTranspose2d(filters[2],filters[1], kernel_size=2, stride=3)  
        self.dec3 = nn.ConvTranspose2d(filters[1],filters[0], kernel_size=3, stride=3)
        self.dec4 = nn.ConvTranspose2d(filters[0] ,1, kernel_size=3, stride=3)
        self.unpool = nn.MaxUnpool2d(2,2,padding=0)

    def forward(self,x):
        x = F.relu(self.enc1(x)) # output will be 168x168x64
        x,indices1 = self.pool(x) #output will be 84x84x64
        x= F.relu(self.enc2(x)) #output will be 28x28x128
        x,indices2 = self.pool(x)   # output will be 14x14x128
        x= F.relu(self.enc3(x)) #output will be 5x5x256
        x= F.relu(self.enc4(x)) #output will be 2x2x512
        Encoded_rep,indices3 = self.pool(x)   # output will be 1x1x512
        x = self.unpool(Encoded_rep,indices3)   # output will be 2x2x512
        x = F.relu(self.dec1(x)) # ouput will be 5x5x256
        x = F.relu(self.dec2(x)) # output will be 14x14x128
        x = self.unpool(x,indices2) #output will be 28x28x128
        x = F.relu(self.dec3(x)) # output will be 84x84x64
        x = self.unpool(x,indices1) #output will be 28x28x128
        x = torch.sigmoid(self.dec4(x)) # output will be 504x504x1
        return x,Encoded_rep.view(-1,512)

**Lets write the first Auto encoder **

In [177]:
net = COV2D_AutoEncoder()
print(net)
if torch.cuda.is_available():
    net.cuda()
summary(net,input_size=(1,504,504),batch_size=1)

COV2D_AutoEncoder(
  (enc1): Conv2d(1, 64, kernel_size=(3, 3), stride=(3, 3))
  (enc2): Conv2d(64, 128, kernel_size=(3, 3), stride=(3, 3))
  (enc3): Conv2d(128, 256, kernel_size=(2, 2), stride=(3, 3))
  (enc4): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dec1): ConvTranspose2d(512, 256, kernel_size=(3, 3), stride=(2, 2))
  (dec2): ConvTranspose2d(256, 128, kernel_size=(2, 2), stride=(3, 3))
  (dec3): ConvTranspose2d(128, 64, kernel_size=(3, 3), stride=(3, 3))
  (dec4): ConvTranspose2d(64, 1, kernel_size=(3, 3), stride=(3, 3))
  (unpool): MaxUnpool2d(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0))
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [1, 64, 168, 168]             640
         MaxPool2d-2  [[-1, 64, 84, 84], [-1, 64, 84, 84]]               0
            Conv2d-3       

In [178]:
train_loader = DataLoader(Data,batch_size=1)

In [179]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
loss_func = nn.MSELoss()
#loss_func = nn.L1Loss()
optimizer = optim.Adam(net.parameters(), lr=0.0001) 
#scheduler = ReduceLROnPlateau(optimizer, 'min')


Training the Auto encoder

In [180]:
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

n_epochs = 100

if torch.cuda.is_available():
    net.cuda()
total_loss = []

epoch_loss = 0
net.train()
for epoch in tqdm(range(n_epochs)):
    print("current epoch is:",epoch)
    epoch_loss = 0
    for x in train_loader:
        mse_loss = 0
        if torch.cuda.is_available():
          x = x.cuda()
        optimizer.zero_grad()
        prediction = net(x)
        SpatialGradient = filters.SpatialGradient()
        loss_der = loss_func(SpatialGradient(prediction[0]), SpatialGradient(x))
        mse_loss = loss_func(prediction[0], x) # <-- note that i'm using the input as the target
        epoch_loss = epoch_loss + mse_loss.item() + loss_der.item()
        (mse_loss+loss_der).backward()
        optimizer.step()
        #scheduler.step(epoch_loss)
        
    #clear_output(wait=True)
    total_loss.append(epoch_loss)
    print('Loss Per epoch: ',epoch_loss)

current epoch is: 0
Loss Per epoch:  8.941986500823987
current epoch is: 1
Loss Per epoch:  8.242584959138185
current epoch is: 2
Loss Per epoch:  7.286656214564573
current epoch is: 3
Loss Per epoch:  6.558523730316665
current epoch is: 4
Loss Per epoch:  6.1162804795312695
current epoch is: 5
Loss Per epoch:  5.671708559268154
current epoch is: 6
Loss Per epoch:  5.310208588256501
current epoch is: 7
Loss Per epoch:  5.0378480845247395
current epoch is: 8
Loss Per epoch:  4.832292100065388
current epoch is: 9
Loss Per epoch:  4.6630941683542915
current epoch is: 10
Loss Per epoch:  4.526108065212611
current epoch is: 11
Loss Per epoch:  4.409322649938986
current epoch is: 12
Loss Per epoch:  4.3008744270773605
current epoch is: 13
Loss Per epoch:  4.206672451691702
current epoch is: 14
Loss Per epoch:  4.126817754993681
current epoch is: 15
Loss Per epoch:  4.0546330966753885
current epoch is: 16


KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt
plt.plot(total_loss)

In [ ]:
net.cpu()
net.eval()
counter = 0
for i in range(30):
    counter = counter + 1
    fig,ax = plt.subplots(1,2,figsize=(8,4))
    x = Data[i]
    input_img = x[0]
    #print(input_img)
    model_pred = net(x.unsqueeze(0))[0].cpu().data.numpy()[0]
    #print(model_pred)
    ax[0].set_title('Input SEM Image',fontsize=12)
    ax[1].set_title('Reconstructed SEM Image',fontsize=12)
    ax[0].imshow(input_img,cmap='gray',vmin=0,vmax=1)
    ax[1].imshow(model_pred,cmap='gray',vmin=0,vmax=1)


Extract Latent representation from Encoder 

Do Clustering and calculate Silhouette Score

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
Attributes = np.empty([len(train_loader),latent_dim])
#print(Attributes.shape)
for i,x in enumerate(Data):
  Temp_results = net(x.unsqueeze(0))
  #print(Temp_results[0])
  Temp_Attributes = (Temp_results[1]).cpu().data.numpy()
  #print("Temp_Attributes:",Temp_Attributes)
  Attributes[i] = Temp_Attributes
  #print(Attributes[i].shape)
#print(Attributes.shape)

In [ ]:
k=20
kmeans = KMeans(n_clusters=k,random_state=0).fit(Attributes)
Clustering_labels = kmeans.labels_
print(Clustering_labels)
Score = silhouette_score(Attributes,Clustering_labels)
print(Score)
sample_score = silhouette_samples(Attributes,Clustering_labels)
#print(sample_score)

In [ ]:
max(sample_score)

Find Optimal K using Silhouette score

In [ ]:
import pandas as pd 
column_names = ["Image","Label","Score"]
df = pd.DataFrame(columns = column_names,index=np.arange(len(Data)))
df['Image'][88]

for i,x in enumerate(Data):
  df['Image'][i] = (x[0]).data.numpy()
  df['Label'][i] = Clustering_labels[i]
  df['Score'][i] = sample_score[i]
  #print(input_img)
  df

In [ ]:
df= df.sort_values('Label',ignore_index=True)
print(df)
df['Image'][120]

In [ ]:
ncols = 7
print(len(Data))
nrows = int(len(Data)/ncols)
figsize = [40, 120]  
fig,ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)
axi = ax.flat
#print(len(axi))
for i,x in enumerate(Data):
  input_img = df['Image'][i]
  axi[i].set_title("label:"+str(df['Label'][i])+" "+"Score:"+str(round(df['Score'][i],2)),fontsize =20)
  axi[i].imshow(input_img,cmap='gray',vmin=0,vmax=1)
fig.tight_layout(pad=0.3)
plt.show

In [ ]:
k_range = 20
Score = np.empty(20)
for k in range(k_range):
  kmeans = KMeans(n_clusters=k+2,random_state=0).fit(Attributes)
  Clustering_labels_k = kmeans.labels_
  Score[k] = silhouette_score(Attributes,Clustering_labels_k)
print(Score)
Max_score_k = np.argmax(Score) + 2
print(Max_score_k)
